<h3>The following implementation of PRADO is taken from this source:<h3>
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://github.com/tensorflow/models/blob/master/research/seq_flow_lite/demo/colab/emotion_colab.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</table>

In [5]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Install Bazel: This will allow us to build custom TensorFlow ops used by the PRADO architecture.

In [3]:
!sudo apt install curl gnupg
!curl https://bazel.build/bazel-release.pub.gpg | sudo apt-key add -
!echo "deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list
!sudo apt update
!sudo apt install bazel=5.4.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.68.0-1ubuntu2.16).
gnupg is already the newest version (2.2.19-3ubuntu2.2).
gnupg set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4714  100  4714    0     0  19240      0 --:--:-- --:--:-- --:--:-- 19240
OK
deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Releas

Install the library:
* `seq_flow_lite` includes the PRADO architecture and custom ops.
* We download the code from GitHub, and then build and install the TF and TFLite ops used by the model.

In [4]:
!git clone https://www.github.com/tensorflow/models
!models/research/seq_flow_lite/demo/colab/setup_workspace.sh
!pip install models/research/seq_flow_lite
!rm -rf models/research/seq_flow_lite/tf_ops
!rm -rf models/research/seq_flow_lite/tflite_ops

Cloning into 'models'...
remote: Enumerating objects: 81861, done.
remote: Counting objects: 100% (328/328), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 81861 (delta 197), reused 261 (delta 157), pack-reused 81533
Receiving objects: 100% (81861/81861), 596.30 MiB | 23.17 MiB/s, done.
Resolving deltas: 100% (58396/58396), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./models/research/seq_flow_lite
  Preparing metadata (setup.py) ... done
  Created wheel for seq-flow-lite: filename=seq_flow_lite-0.1-py3-none-any.whl size=758969 sha256=6431614366f98ec7cdd0798861e636cd15521648fd4981013a0bf9e8aea113d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-86hpp65n/wheels/eb/52/28/00d5797353c4a3a35688718d809124dd994ea8487cb7676fba
Successfully built seq-flow-lite


In [6]:
%cd /content/models/research/seq_flow_lite

/content/models/research/seq_flow_lite


labels which the multi labelled prado model.

In [7]:
LABELS = [
    'admiration',
    'amusement',
    'anger',
    'annoyance',
    'approval',
    'caring',
    'confusion',
    'curiosity',
    'desire',
    'disappointment',
    'disapproval',
    'disgust',
    'embarrassment',
    'excitement',
    'fear',
    'gratitude',
    'grief',
    'joy',
    'love',
    'nervousness',
    'optimism',
    'pride',
    'realization',
    'relief',
    'remorse',
    'sadness',
    'surprise',
    'neutral',
]

MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': True,
    'quantize': False,
    'max_seq_len': 128,
    'max_seq_len_inference': 128,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 512,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,
    'doc_size_levels': 0,
    'add_bos_tag': False,
    'add_eos_tag': False,
    'pre_logits_fc_layers': [],
    'text_distortion_probability': 0.0,
}

In [8]:
from layers import base_layers
from models import prado
from layers import projection_layers

def build_model(mode):
  # First we define our inputs.
  inputs = []
  if mode == base_layers.TRAIN or mode == base_layers.EVAL:
    # For TRAIN and EVAL, we'll be getting dataset examples,
    # so we'll get projections and sequence_lengths.
    projection = tf.keras.Input(
        shape=(MODEL_CONFIG['max_seq_len'], MODEL_CONFIG['feature_size']),
        name='projection',
        dtype='float32')

    sequence_length = tf.keras.Input(
        shape=(), name='sequence_length', dtype='float32')
    inputs = [projection, sequence_length]
  else:
    # Otherwise, we get string inputs which we need to project.
    input = tf.keras.Input(shape=(), name='input', dtype='string')
    projection_layer = projection_layers.ProjectionLayer(MODEL_CONFIG, mode)
    projection, sequence_length = projection_layer(input)
    inputs = [input]

  # Next we add the model layer.
  model_layer = prado.Encoder(MODEL_CONFIG, mode)
  logits = model_layer(projection, sequence_length)

  # Finally we add an activation layer.
  if MODEL_CONFIG['multilabel']:
    activation = tf.keras.layers.Activation('sigmoid', name='predictions')
  else:
    activation = tf.keras.layers.Activation('softmax', name='predictions')
  predictions = activation(logits)

  model = tf.keras.Model(
      inputs=inputs,
      outputs=[predictions])
  
  return model

Loading the model for model weights and running inference for the selected examples:

In [9]:
model = build_model(base_layers.PREDICT)
model.load_weights('/content/gdrive/MyDrive/model_weights/model_checkpoint')

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Training a KMeans clustering algorithm with 250 top [Hollywood movies](https://www.imdb.com/chart/top-english-movies/)

In [11]:
%cd /content/gdrive/MyDrive/movie_recommender

/content/gdrive/MyDrive/movie_recommender


In [12]:
import numpy as np
import pandas as pd
from imdb_scraper import *
from recommender import reviews_to_sent , get_mov_ranked , preprocess , GENRES

In [13]:
# Create a pandas dataframe containing the movie and genres.
df = pd.read_csv('movies.csv')

# representing the genres of each movie in a sparse 
df = df.assign(**{genre: 0 for genre in GENRES})

In [14]:
df = df.sample(500)

In [ ]:
preprocessed_df = preprocess(df, 'preprocessed_df.csv', model, tokenized=True, count=5)

 76%|███████▋  | 382/500 [45:38<15:40,  7.97s/it]

In [16]:
mask = preprocessed_df['sentiment'].apply(lambda x: isinstance(x, np.ndarray) and x.shape == (28,))
preprocessed_df = preprocessed_df[mask]

Movie Recommendation for the 2021 Movie [Nobody](https://www.imdb.com/title/tt7888964/?ref_=tt_urv) and similarity between sentiment labels of prediction and movies from the vector space using cosine similarity metric

In [17]:
movie="Nobody"

In [18]:
recommended_movies = get_mov_ranked(preprocessed_df , movie , model , tokenizer=None , mov_count=10)

In [19]:
print("Recommended Movies: ")
for mov in recommended_movies:
  print(mov)

Recommended Movies: 
47  Ronin (Shijûshichinin no shikaku) (1994)
Black Rain (1989)
Paris by Night (2012)
Brother (Brat) (1997)
The Gun Runners (1958)
People I Know (2002)
History of a Crime (1901)
The Villainess (2017)
Hell's Highway (1932)
Trespass Against Us (2017)
